![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [1]:
import pandas as pd
import numpy as np

# Start coding here...

df = pd.read_csv("bank_marketing.csv")

print(df.columns)
print('--------------------')

# -----------------------------------------------------------------------------------

df['job'] = df['job'].replace(".","_")

df['education'] = df['education'].str.replace(".","_")
df['education'] = df['education'].replace("unknown",np.NaN)

df['credit_default'] = np.where(df['credit_default'] == 'yes', 1, 0)
df['credit_default'] = df['credit_default'].astype(bool)

df['mortgage'] = np.where(df['mortgage'] == 'yes', 1, 0)
df['mortgage'] = df['mortgage'].astype(bool)

client = df[['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'mortgage']].copy()
client.to_csv('client.csv', index=False)

print(client.dtypes)
print('--------------------')

# -----------------------------------------------------------------------------------

df['previous_outcome'] = np.where(df['previous_outcome'] == 'success', 1, 0)
df['previous_outcome'] = df['previous_outcome'].astype(bool)

df['campaign_outcome'] = np.where(df['campaign_outcome'] == 'yes', 1, 0)
df['campaign_outcome'] = df['campaign_outcome'].astype(bool)

df['last_contact_date'] =  '2022-' + df['month'].astype(str) + '-' + df['day'].astype(str)
df['last_contact_date'] = pd.to_datetime(df["last_contact_date"])

campaign = df[['client_id', 'number_contacts', 'contact_duration', 'previous_campaign_contacts', 'previous_outcome', 'campaign_outcome', 'last_contact_date']].copy()
campaign.to_csv('campaign.csv', index=False)

print(campaign.dtypes)
print('--------------------')

# -----------------------------------------------------------------------------------

economics = df[['client_id', 'cons_price_idx', 'euribor_three_months']].copy()
economics.to_csv('economics.csv', index=False)

print(economics.dtypes)
print('--------------------')


Index(['client_id', 'age', 'job', 'marital', 'education', 'credit_default',
       'mortgage', 'month', 'day', 'contact_duration', 'number_contacts',
       'previous_campaign_contacts', 'previous_outcome', 'cons_price_idx',
       'euribor_three_months', 'campaign_outcome'],
      dtype='object')
--------------------
client_id          int64
age                int64
job               object
marital           object
education         object
credit_default      bool
mortgage            bool
dtype: object
--------------------
client_id                              int64
number_contacts                        int64
contact_duration                       int64
previous_campaign_contacts             int64
previous_outcome                        bool
campaign_outcome                        bool
last_contact_date             datetime64[ns]
dtype: object
--------------------
client_id                 int64
cons_price_idx          float64
euribor_three_months    float64
dtype: object
----------

In [2]:
df = pd.read_csv("bank_marketing.csv")

print(df.dtypes)

for col in df:
    print(col)
    print("--------------")
    print(df[col].value_counts())



client_id                       int64
age                             int64
job                            object
marital                        object
education                      object
credit_default                 object
mortgage                       object
month                          object
day                             int64
contact_duration                int64
number_contacts                 int64
previous_campaign_contacts      int64
previous_outcome               object
cons_price_idx                float64
euribor_three_months          float64
campaign_outcome               object
dtype: object
client_id
--------------
0        1
27512    1
27454    1
27455    1
27456    1
        ..
13730    1
13731    1
13732    1
13733    1
41187    1
Name: client_id, Length: 41188, dtype: int64
age
--------------
31    1947
32    1846
33    1833
36    1780
35    1759
      ... 
89       2
91       2
94       1
87       1
95       1
Name: age, Length: 78, dtype: int64
job
-------